# Данные

In [15]:
import csv
from collections import defaultdict, Counter
import random
import json
from metrics import apk
random.seed(42)

In [2]:
from sklearn.ensemble import RandomForestRegressor
import numpy as np

In [3]:
import scipy as sp
import scipy.sparse
import scipy.sparse.linalg

In [4]:
#Словари для основной выборки
user_to_items = defaultdict(set)
item_to_users = defaultdict(set)
edges = []
#Словари для тестовой выборки
test_user_to_items = defaultdict(set)
test_item_to_users = defaultdict(set)
test_edges = []

with open("data/train_likes.csv") as datafile:
    for like in csv.DictReader(datafile):
        # Кидаем монетку. В зависимости от результата кладём в обучение или тест
        if random.random() < 0.90:
            user_to_items[like['user_id']].add(like['item_id'])
            item_to_users[like['item_id']].add(like['user_id'])
            edges.append((like['user_id'], like['item_id']))
        else:
            test_user_to_items[like['user_id']].add(like['item_id'])
            test_item_to_users[like['item_id']].add(like['user_id'])
            test_edges.append((like['user_id'], like['item_id']))

In [5]:
all_items = set(item_to_users.keys()) | set(test_item_to_users.keys())
all_users = set(user_to_items.keys()) | set(test_user_to_items.keys())

In [6]:
users_list = list(user_to_items.keys())
test_users_list = list(test_user_to_items.keys())

In [7]:
user_to_i = {user: i for i, user in enumerate(all_users)}
item_to_i = {item: i for i, item in enumerate(all_items)}
all_users_list = list(all_users)
all_items_list = list(all_items)

In [8]:
matrix = sp.sparse.lil_matrix((len(all_users), len(all_items)))
test_matrix = sp.sparse.lil_matrix((len(all_users), len(all_items)))
for user, items in user_to_items.items():
    for item in items:
        matrix[user_to_i[user], item_to_i[item]] = True
for user, items in test_user_to_items.items():
    for item in items:
        test_matrix[user_to_i[user], item_to_i[item]] = True
matrix = matrix.tocsr()
test_matrix = test_matrix.tocsr()

In [9]:
matrix

<55863x23891 sparse matrix of type '<class 'numpy.float64'>'
	with 99582 stored elements in Compressed Sparse Row format>

In [10]:
u, s, vt = sp.sparse.linalg.svds(matrix.astype(np.float32), k=100)

In [11]:
u, s, vt

(array([[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [ -5.78052363e-12,  -1.12967682e-11,   4.27958224e-11, ...,
          -1.46805983e-12,   1.39074178e-11,   5.32049231e-12],
        ..., 
        [  6.98581338e-03,  -1.19996611e-02,  -1.05127774e-03, ...,
          -2.49119385e-05,  -2.29380283e-04,   2.38834444e-04],
        [  1.57555600e-03,   1.87347014e-03,  -9.51645314e-04, ...,
           7.12322770e-04,  -5.07251010e-04,   6.79163903e-04],
        [  7.99124222e-03,   7.36495899e-03,   1.09396344e-02, ...,
          -1.01648846e-04,  -1.04412588e-03,   1.13458082e-03]], dtype=float32),
 array([ 10.72060871,  10.74181747,  10.77246761,  10.81643581,
         10.87412643,  10.89381695,  10.9067688 ,  10.93737125,
         11.00072002,  11.02944088,  11.04997444,

In [12]:
films = json.load(open('data/items.json'))
films = {a['id']:a for a in films}
for a in films.values():
    del a['id']

In [13]:
for f in films.values():
    if 'genre' in f:
        f[f['genre']] = 1
        del f['genre']

In [16]:
features = Counter()
for f in films.values():
    features.update(set(f.keys()))

In [17]:
features.most_common(20)

[('duration', 138078),
 ('year', 138078),
 (0, 34128),
 (3, 32154),
 ('f_114306', 30650),
 (1, 28635),
 ('f_79251', 16647),
 ('f_117573', 15295),
 ('f_138980', 15215),
 (5, 15008),
 ('f_64513', 14020),
 ('f_205162', 10693),
 ('f_122038', 10436),
 ('f_84602', 8377),
 ('f_68894', 8348),
 ('f_127793', 7946),
 ('f_72071', 7866),
 (4, 7386),
 ('f_210900', 7085),
 ('f_130202', 6371)]

In [18]:
small_features_size = 20 

In [19]:
small_features = set([_[0] for _ in features.most_common(small_features_size)])

In [20]:
feature_to_i = {feature: i for i, feature in enumerate(small_features)}

In [23]:
feature_to_i

{0: 0,
 1: 1,
 3: 3,
 4: 4,
 5: 5,
 'f_127793': 6,
 'f_79251': 7,
 'f_117573': 2,
 'f_114306': 9,
 'f_122038': 10,
 'f_205162': 11,
 'duration': 12,
 'f_68894': 13,
 'f_72071': 14,
 'f_130202': 8,
 'year': 15,
 'f_210900': 16,
 'f_84602': 17,
 'f_138980': 18,
 'f_64513': 19}

# Фильтрация пользователей
* Значительная часть пользователей имеет всего 1-2 просмотра. При всём желании, рекоммендовать им что-либо осмысленное при помощи рассматриваемого здесь метода мы вряд ли сможем. Для простоты вычислений, удалим их из выборки.
* Важно понимать, что качество на оставшихся пользователях скорее всего будет выше, чем на первоначальной выборке.

In [24]:
min_items_per_user = 2
from copy import copy
for user in copy(test_user_to_items).keys():
    
    n_items_per_user = len(user_to_items[user]) + len(test_user_to_items[user])
    
    if n_items_per_user <= min_items_per_user:
        del user_to_items[user]
        del test_user_to_items[user]

### Рекоммендующая функция
Позволим себе немного вольности: наша функция будет возвращать не вероятности, а список фильмов в порядке убывания "рекомендованности".

* Рекоммендованность фильма item пользователю user посчитаем так:
  * Для каждого фильма, полайканного пользователем user, найдём других людей, которым понравился фильм.
  * Сложим всех таких "друзей по лайкам" вместе и назовём соседями (__neighborhood__) пользователя.
  * Для фильма item узнаем его аудиторию - множество пользователей, которые его лайкнули
  * Пригодность фильма пользователю - то, насколько "друзьям по лайкам" пользователя нравится этот фильм.

Для примера, будем использовать косинусную меру расстояния
  
$ cos(u_{film}, u_{neighborhood}) = $ =$ u_{film} \cdot u_{neighborhood} \over |u_{film}| |u_{neighborhood}| $


$u_{neighborhood}$ зависит только от пользователя, но не от фильма, поэтому при сравнении фильмов по пригодности для одного пользователя, его можно исключить из формулы для простоты вычислений.

$ similarity(u_{film}, u_{neighborhood}) = $ $  u_{film} \cdot u_{neighborhood} \over |u_{film}| $
  
  
Распишем формулу подробно:

$ similarity(u_{film}, u_{neighborhood}) = $ $ \sum _{u_i} [u_i \in u_{film}] \cdot [u_i \in u_{neighborhood}] \over |u_{film}|  $

* u_i - очередной пользователь (в цикле по всем пользователям)
  
Выражение $[u_i \in u_{neighborhood}]$ здесь означает "сколько раз очередной пользователь входит в множество друзей по лайкам"
  
  

In [25]:
from math import sqrt
from collections import Counter

def recommend(user, n_best = 10):
    user_items = user_to_items[user]
    
    neighborhood = Counter()
    for item in user_items:
        neighborhood.update(item_to_users[item])
    
    #словарь {фильм -> пригодность фильма пользователю}
    item_similarities = {}
    
    for item in all_items:
        if item in user_items: continue
        item_users = item_to_users[item]
        if len(item_users) == 0: continue
        
        n_common_users = sum(neighborhood[user] for user in item_users)
        similarity = float(n_common_users) / sqrt(len(item_users))
        item_similarities[item] = similarity
    
    items_sorted = sorted(all_items, key = lambda x: item_similarities.get(x, 0),reverse = True)
    
    return items_sorted[:n_best]

In [26]:
user_to_int = dict()
for i, user in enumerate(all_users):
    user_to_int[user] = i 

In [27]:
#Making dataset
user_features = defaultdict(lambda:defaultdict(lambda:0))
for user in list(user_to_items.keys())[0:]:
    for item in user_to_items[user]:
        if item in films:
            for feature, value in films[item].items():    
                user_features[user][feature]+=value
    for f in user_features[user]:
        user_features[user][f]/=len(user_to_items)

In [28]:
edges[0] in test_edges

False

In [29]:
def extract_features(user=None, item=None, X_sample = np.zeros(2*small_features_size, dtype='float32')):
    if user is not None:
         for f in user_features[user].keys()&small_features:
            X_sample[2*feature_to_i[f]] = user_features[user][f]
    if item is not None and item in films:
        for f in films[item].keys()&small_features:
            X_sample[2*feature_to_i[f]+1] = films[item][f]
    return X_sample

In [30]:
len(edges)

100068

In [31]:
def generate_random_samples(X_size = 100, use_test=False):
    X = np.zeros((X_size, 2*small_features_size), dtype='float32')
    Y = np.zeros(X_size, dtype='int8')
    if use_test:
        local_users_list = test_users_list
        local_user_to_items = test_user_to_items
    else:
        local_users_list = users_list
        local_user_to_items = user_to_items
    for i in range(X_size):
        film =''
        while film not in films: 
            result = random.random()
            if result > 0.50:
                result = 1
            else:
                result = 0
            user = random.choice(local_users_list)
            if result==0:
                film = random.choice(all_items_list)
                if film in local_user_to_items[user]:
                    result = 1
            else:
                film = random.choice(list(local_user_to_items[user]|{''}))
        X[i] = extract_features(user, film)
        Y[i] = result
    return X, Y

In [32]:
def generate_samples(X_true = 100,  use_test=False):
    X_fake=X_true
    X = np.zeros((X_true+X_fake, 2*small_features_size), dtype='float32')
    Y = np.zeros(X_true+X_fake, dtype='int8')
    if use_test:
        local_users_list = test_users_list
        local_user_to_items = test_user_to_items
        local_edges = test_edges 
    else:
        local_edges = edges 
        local_users_list = users_list
        local_user_to_items = user_to_items
    for i, (user, film)  in enumerate(local_edges[:X_true]):
        X[i] = extract_features(user, film)
        Y[i] = 1
    for i in range(X_true, X_true+X_fake):
        user = random.choice(local_users_list)
        film = random.choice(all_items_list)
        X[i] = extract_features(user, film)
        if film in local_user_to_items[user]:
            Y[i] = 1
    return X, Y

In [53]:
X, Y = generate_samples(100000)

In [33]:
rf = RandomForestRegressor(n_estimators=200, n_jobs=4)

In [54]:
rf.fit(X, Y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=200, n_jobs=4, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [70]:
X, Y = generate_random_samples(100, use_test=False)
print("Train score: %s" %(rf.score(X, Y),))
X, Y = generate_samples(100, use_test=True)
print("Test score: %s" %(rf.score(X, Y),))

Train score: -0.118441427723
Test score: -0.654078082774


In [71]:
X, Y = generate_random_samples(10, use_test=False)
rf.predict(X), Y

(array([ 0.94 ,  0.93 ,  0.935,  0.945,  0.945,  0.93 ,  0.945,  0.52 ,
         0.69 ,  0.315]), array([0, 1, 1, 0, 1, 0, 0, 1, 1, 1], dtype=int8))

In [258]:
def user_film_mk3(user, item, debug=False):
    X = np.zeros((1, 2*small_features_size), dtype='float32')
    X[0] = extract_features(user, item)
    probs = rf.predict(X)
    if debug:
        print(probs)
    return probs[0]

In [30]:
def user_film_mk1(user, item, debug=False):
    #print(len(user_features[user]))
    cursum = 0
    curcnt = 0
    if item in films:
        for feature, value in films[item].items():
            if type(value) is int:
                curcnt+=1    
                cursum+=value*user_features[user][feature]
    if curcnt==0:
        curcnt+=1
    #cursum/=curcnt
    if debug:
        print(cursum)
    return cursum

In [31]:
def user_film_mk2(user, item, debug=False):
    ui = user_to_i[user]
    ii = item_to_i[item]
    if debug:
        print(ui, ii)
    a = np.dot(u[ui,:] * s, vt[:,ii])
    return a

In [173]:
user_film_mk3('26052b20aa96ed8d803dbfe4e9497192', '45ea2aa2143effcb6575daf0143e31b4', debug=True)

[[ 1.  0.]]


1.0

In [257]:
def recommend_mk3_manual(user, n_best = 10, debug=False):
    user_items = user_to_items[user]

    item_similarities = {}
    cur_items = [item for item in all_items if item not in user_items and len(item_to_users[item])>0]
    
    X = np.zeros((len(cur_items), 2*small_features_size), dtype='float32')
    X_user = extract_features(user)
    for i, item in enumerate(cur_items):
        X[i] = extract_features(user=None, item=item, X_sample=X_user)
    
    probs =rf.predict(X)
    item_similarities = {key: prob for key, prob in zip(cur_items, probs)}
    items_sorted = sorted(all_items, key = lambda x: item_similarities.get(x, 0),reverse = True)
    if debug:
        print(X)
        for a in  items_sorted[:]:
            print((a, item_similarities.get(a, 0)))
    return items_sorted[:n_best]

In [245]:
def generic_recommend(user_flim_function):
    def recommend(user, n_best = 10, debug=False):
        user_items = user_to_items[user]

        item_similarities = {}
        for item in all_items:
            if item in user_items: continue
            item_users = item_to_users[item]
            if len(item_users) == 0: continue

            item_similarities[item] = user_flim_function(user, item)

        items_sorted = sorted(all_items, key = lambda x: item_similarities.get(x, 0),reverse = True)
        if debug:
            for a in  items_sorted[:n_best]:
                print((a, item_similarities.get(a, 0)))
        return items_sorted[:n_best]
    return recommend

In [37]:
generic_recommend(user_film_mk2)('1a337111f63f6a7f86cebf6b2ad3d732', debug=True)
user_film_mk2('1a337111f63f6a7f86cebf6b2ad3d732', 'c745ed11b94ed93f3008897c75240de3', debug=True)

('7682d6641caf183ffcf2616c9bb1e434', 0.087393582)
('b5e424608d041f38c2fd6125b3bcd40f', 0.062320966)
('006a602774c79d476cdbd5dff7dd24f1', 0.047886718)
('e9caaa12a277abf87ba07b84e09a4170', 0.04556052)
('aa5f2ca699da42e467e550f9f071fb3f', 0.044249821)
('bcee3a68dbfb6b4ea97ac53ceb1d3287', 0.044186711)
('e8047b4262e676d28f50816ac3fde1ca', 0.042720918)
('77c7998b3d3d41f3fc3aec786b4015ac', 0.038098734)
('1da6f84598fbeea050da49fe37297932', 0.037306763)
('2dc12b93072f94bca9438902bddc36c4', 0.035170272)
24171 22183


0.0024644737

In [38]:
generic_recommend(user_film_mk1)('1a337111f63f6a7f86cebf6b2ad3d732', debug=True)
user_film_mk1('1a337111f63f6a7f86cebf6b2ad3d732', 'c745ed11b94ed93f3008897c75240de3', debug=True)

('e13df82b3252eaeefc0c13599695f9c7', 0.0006581965414763549)
('634e622e5c957c2cd8ddb98ca75ef937', 0.0005584697927678162)
('8faa6327a2ca2eb92e738ffc204d35c0', 0.0005584697927678162)
('64b6536649e20bd72d7183ec717b6c86', 0.0005385244430261086)
('ef786a839c609561b76ce03768eace80', 0.0005185790932844007)
('bcee3a68dbfb6b4ea97ac53ceb1d3287', 0.0005185790932844007)
('5c6d1862dc1e82f340962b308999e603', 0.0005185790932844007)
('ff987a60d68f5bbbefe97f8c5062711a', 0.0004986337435426931)
('753bce7ac492e6c95b07b85f8877cf5a', 0.00047868839380098526)
('05dff6d4da204f552d7e8d1a18c31936', 0.0004587430440592776)
0


0

In [247]:
recommend_mk3_manual('a18f526db904f8f2ee4c8d178bfc818c', debug=True)

[[  1.99119895e-05   0.00000000e+00   0.00000000e+00 ...,   8.26173723e-01
    0.00000000e+00   0.00000000e+00]
 [  1.99119895e-05   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  1.99119895e-05   0.00000000e+00   0.00000000e+00 ...,   8.14270210e+00
    0.00000000e+00   0.00000000e+00]
 ..., 
 [  1.99119895e-05   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  1.99119895e-05   0.00000000e+00   0.00000000e+00 ...,  -7.35781789e-01
    0.00000000e+00   0.00000000e+00]
 [  1.99119895e-05   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]]
('5868853865709a004ec25f2f61b86bde', 0.97499999999999998)
('97197447a4562c7ff88410b145f7c907', 0.96999999999999997)
('f423101eabc9565aad1f6d01fd651891', 0.96999999999999997)
('4872bb21e0a268161ab82f6b59ae5aa3', 0.96499999999999997)
('96173cc1dc01533ad68e1ae98291c24f', 0.96499999999999997)
('f4a7b753e9c336ab047129d5ca40

['5868853865709a004ec25f2f61b86bde',
 '97197447a4562c7ff88410b145f7c907',
 'f423101eabc9565aad1f6d01fd651891',
 '4872bb21e0a268161ab82f6b59ae5aa3',
 '96173cc1dc01533ad68e1ae98291c24f',
 'f4a7b753e9c336ab047129d5ca40885e',
 '4d50ae179d81601f161d6c6cc181e2b8',
 '30d69ba3457fbe63ddf07f88adbca502',
 '777e3da0f861706fd2f32ba6eabe66bc',
 '7729af02116e3e75deea642d871e88a7']

In [238]:
list(test_user_to_items.items())[7]

('a18f526db904f8f2ee4c8d178bfc818c', {'7a040062b953a9a3e7fb1545def5773f'})

In [42]:
def recommend_dummy(user, n_best = 10):
    item_similarities = []
    for item in all_items:
        #пропустим те фильмы, которые пользователь уже просмотрел, если нас об этом попросили
        if item in user_to_items[user]: continue
        item_similarities.append(item)
         
    random.shuffle(item_similarities)
    #вернём n_best наиболее пригодных
    #print(items_sorted[:n_best])
    return item_similarities[:n_best]

# Оценка качества - map@k

In [45]:
check_quality(recommend_dummy,10,200)

0 / 200
100 / 200
AP@10 = 9.259259259259259e-05


In [167]:
check_quality(recommend_mk3_manual,100, 200)

0 / 200
100 / 200
AP@100 = 0.0


In [139]:
check_quality(generic_recommend(user_film_mk1),10, 200)

0 / 200
100 / 200
AP@5 = 0.0125


In [55]:
check_quality(generic_recommend(user_film_mk2),5,200)

0 / 200
100 / 200
AP@5 = 0.026083333333333333


In [1]:
check_quality(recommend,5, 200)

NameError: name 'check_quality' is not defined

In [49]:
len(test_user_to_items)

2732

In [135]:
#сколько рекоммендаций рассматриваем
def check_quality(function, K = 10, max_n_users = len(test_user_to_items)):
    APatK_per_user = []
    user_list = list(test_user_to_items.keys())[:max_n_users]
    
    for i, user in enumerate(user_list):
        #фильмы, которые пользователю на самом деле нравятся
        test_items = test_user_to_items[user]

        #Выдать топ-K рекоммендаций
        recommendation_list = function(user,n_best=K)
        #Посчитать ap@k
        user_APatK = apk(test_items, recommendation_list,k=K)

        #и сложить в коробку
        APatK_per_user.append(user_APatK)

        #Progress bar
        if i % 100 ==0:
            print(i,'/',max_n_users)

        if i > max_n_users:
            break

    print('AP@{} = {}'.format(K, sum(APatK_per_user)/len(APatK_per_user)))


# Notes
* Кроме качества рекоммендаций, map@k ещё зависит от доли тестовой выборки, фильтрации и от самого K. Сравнивать качество разных алгоритмов имеет смысл только при одинаковом K и тестовой выборке.
* Давать полезные рекоммендации пользователям с малым числом просмотров тоже можно: например, можно выдавать наиболее популярные в целом фильмы.
* Разделение на обучение/тест честнее делать на по времени: первые 70% (например) лайков в обучение, остальные в тест. Это ближе к реальной жизни, когда вы сначала обучаете модель на логах, а потом применяете на новых сессиях пользователей.